In [1]:
def load_data(path, test=False):
    lines = []
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        lines = [l.strip() for l in lines]
    if test:
        ids = [l.split(',')[0] for l in lines]
        return ids, lines
    else:
        return lines

In [2]:
pos = load_data('data/train_pos.txt')
neg = load_data('data/train_neg.txt')

In [3]:
import string

def clean(tweet):
    lower = tweet.lower()
    no_user_token = lower.replace('<user>', '')
    no_token = no_user_token.replace('<url>', '')
    no_punct = no_token.translate(str.maketrans('', '', string.punctuation))
    no_digit = [w for w in no_punct.split() if not w.isdigit()]
    return no_digit

In [4]:
clean_pos = [clean(t) for t in pos]
clean_neg = [clean(t) for t in neg]

In [5]:
from nltk.stem.wordnet import WordNetLemmatizer


import nltk
nltk.download('wordnet')

def lemmatize(words):
    lem = WordNetLemmatizer()
    verbs = [lem.lemmatize(w, 'v') for w in words] # verbs
    return [lem.lemmatize(w, 'n') for w in verbs] # nouns

[nltk_data] Downloading package wordnet to /home/anton/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
lemmatized_pos = [lemmatize(w) for w in clean_pos]
lemmatized_neg = [lemmatize(w) for w in clean_neg]

In [7]:
def save_cleaned(path, data, isPos):
    label = "-1"
    if isPos :
        label = "1"
    with open(path, 'w', encoding='utf-8') as f:
        for words in data:
            f.write(' '.join(words) + " , " + label)
            f.write('\n')
            
def append_cleaned(path, data, isPos):
    label = "0"
    if isPos :
        label = "1"
    with open(path, 'a', encoding='utf-8') as f:
        for words in data:
            mot = ' '.join(words)
            if mot != "":
                f.write(mot + " , " + label)
                f.write('\n')

In [8]:
!rm data/clean_train.txt

In [9]:
save_cleaned('data/clean_train_pos.txt', lemmatized_pos, True)
save_cleaned('data/clean_train_neg.txt', lemmatized_neg, False)


append_cleaned('data/clean_train.txt', lemmatized_pos, True)
append_cleaned('data/clean_train.txt', lemmatized_neg, False)

-----------------------------------------------

In [9]:
from collections import Counter
counter_lem_pos = Counter()
counter_lem = Counter()
for t in lemmatized_pos:
    for w in t:
        counter_lem_pos[w]+=1
        counter_lem[w]+=1
counter_lem_neg = Counter()
for t in lemmatized_neg:
    for w in t:
        counter_lem_neg[w]+=1
        counter_lem[w]+=1

In [12]:
clean_lem_counter = {x : counter_lem[x] for x in counter_lem if counter_lem[x] > 50}
len(clean_lem_counter)

3330

In [13]:
# most common in both
thresh = 100
most_common_pos = [w for w,c in counter_lem_pos.most_common()[:thresh]]
most_common_neg = [w for w,c in counter_lem_neg.most_common()[:thresh]]

In [14]:
most_common_in_both = []
for w in most_common_pos:
    if w in most_common_neg:
        most_common_in_both.append(w)
#most_common_in_both

In [15]:
# remove most common in both
clean_lem_counter_2 = counter_lem.copy()
for w in most_common_in_both:
    clean_lem_counter_2.pop(w)

In [16]:
# remove words that appear scarcely
thresh_2 = 50
chosen_words = [x for x in clean_lem_counter_2 if clean_lem_counter_2[x] > thresh_2]
len(chosen_words)

3255

In [17]:
# save words
with open('data/chosen_words.txt', 'w', encoding='utf-8') as f:
    for w in chosen_words:
        f.write(w + '\n')

---------------------------------------------------

In [47]:
import numpy as np
import random

training = np.genfromtxt('data/clean_train.txt', delimiter=' , ', dtype=None)

random.shuffle(training)

train_data = training[:140000]
test_data = training[140000:]


# create our training data from the tweets
train_x = [x[0] for x in train_data]
# index all the sentiment labels
train_y = np.asarray([x[1] for x in train_data])

test_x = [x[0] for x in test_data]

test_y = np.asarray([x[1] for x in test_data])


/home/anton/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  after removing the cwd from sys.path.


In [48]:
import json
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer

# only work with the 3000 most popular words found in our dataset
max_words = 3000

# create a new Tokenizer
tokenizer = Tokenizer(num_words=max_words)
# feed our tweets to the Tokenizer
tokenizer.fit_on_texts(train_x)

# Tokenizers come with a convenient list of words and IDs
dictionary = tokenizer.word_index
# Let's save this out so we can use it later
with open('dictionary.json', 'w') as dictionary_file:
    json.dump(dictionary, dictionary_file)


def convert_text_to_index_array(text):
    # one really important thing that `text_to_word_sequence` does
    # is make all texts the same length -- in this case, the length
    # of the longest text in the set.
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]

allWordIndices = []
# for each tweet, change each token to its ID in the Tokenizer's word_index
for text in train_x:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

# now we have a list of all tweets converted to index arrays.
# cast as an array for future usage.
allWordIndices = np.asarray(allWordIndices)

# create one-hot matrices out of the indexed tweets
train_x = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')
# treat the labels as categories
train_y = keras.utils.to_categorical(train_y, 2)

In [49]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

model = Sequential()
model.add(Dense(512, input_shape=(max_words,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [50]:
model.compile(loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'])

In [52]:
model.fit(train_x, train_y,
  batch_size=32,
  epochs=2,
  verbose=1,
  validation_split=0.1
)

Train on 126000 samples, validate on 14000 samples
Epoch 1/2
126000/126000 [==============================] - 70s 552us/step - loss: 0.0925 - accuracy: 0.9793 - val_loss: 0.6997 - val_accuracy: 0.8001
Epoch 2/2
126000/126000 [==============================] - 68s 537us/step - loss: 0.0737 - accuracy: 0.9820 - val_loss: 0.6926 - val_accuracy: 0.8066


In [53]:
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')

In [ ]:
import json
import numpy as np
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json

# we're still going to use a Tokenizer here, but we don't need to fit it
tokenizer = Tokenizer(num_words=3000)
# for human-friendly printing
labels = ['negative', 'positive']

# read in our saved dictionary
with open('dictionary.json', 'r') as dictionary_file:
    dictionary = json.load(dictionary_file)

# this utility makes sure that all the words in your input
# are registered in the dictionary
# before trying to turn them into a matrix.
def convert_text_to_index_array(text):
    words = kpt.text_to_word_sequence(text)
    wordIndices = []
    for word in words:
        if word in dictionary:
            wordIndices.append(dictionary[word] + 1)
        else:
            wordIndices.append(1)
            #print("'%s' not in training corpus; ignoring." %(word))
    return wordIndices

def predict(test_data):
    
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()

    model = model_from_json(loaded_model_json)

    model.load_weights('model.h5')
    
    preds = []
    
    test_data = [' '.join(lemmatize(clean(w))) for w in test_data]
    
    for l in test_data:
        testArr = convert_text_to_index_array(l)
        input = tokenizer.sequences_to_matrix([testArr], mode='binary')
        
        pred = model.predict(input)
        print(pred)
        
        preds.append(np.argmax(pred))
    return preds

#test_ids, test_data = load_data('data/test_data.txt', test=True)
preds = predict(test_x)

[[0.00794091 0.9920591 ]]
[[0.00731731 0.99268275]]
[[0.00198052 0.99801946]]
[[0.00125712 0.9987429 ]]
[[0.00235111 0.9976489 ]]
[[0.00183214 0.9981679 ]]
[[0.10529619 0.8947038 ]]
[[0.00215735 0.99784267]]
[[8.6846744e-04 9.9913150e-01]]
[[0.07398175 0.9260183 ]]
[[0.00242816 0.9975719 ]]
[[0.00156185 0.9984382 ]]
[[0.04084443 0.9591556 ]]
[[0.8319194  0.16808066]]
[[0.01405821 0.9859418 ]]
[[0.04648447 0.9535155 ]]
[[0.00658965 0.9934103 ]]
[[0.00256555 0.99743444]]
[[0.00336264 0.99663734]]
[[0.00188588 0.9981141 ]]
[[0.01090446 0.9890955 ]]
[[0.00474783 0.99525213]]
[[0.15423287 0.84576714]]
[[0.00474185 0.99525815]]
[[0.0773844 0.9226156]]
[[0.00549505 0.994505  ]]
[[0.03479351 0.9652065 ]]
[[0.00633846 0.9936615 ]]
[[0.1480322 0.8519678]]
[[0.00186533 0.9981346 ]]
[[0.00500016 0.9949999 ]]
[[0.02193309 0.9780669 ]]
[[0.00288942 0.9971106 ]]
[[0.00791824 0.99208176]]
[[0.0018648 0.9981352]]
[[0.00261889 0.99738103]]
[[9.1880700e-04 9.9908113e-01]]
[[0.00534876 0.99465126]]
[[0.00

[[0.00386653 0.99613345]]
[[0.00178317 0.9982168 ]]
[[0.00163743 0.9983626 ]]
[[0.00317524 0.99682474]]
[[0.0027544  0.99724567]]
[[0.00150513 0.99849486]]
[[0.00135847 0.99864155]]
[[0.02895717 0.9710428 ]]
[[0.00107915 0.9989209 ]]
[[8.9106860e-04 9.9910897e-01]]
[[0.01197919 0.98802084]]
[[0.00128079 0.9987192 ]]
[[9.3461043e-04 9.9906546e-01]]
[[0.00197698 0.99802303]]
[[0.00486824 0.99513173]]
[[0.00365397 0.99634606]]
[[0.00435942 0.9956405 ]]
[[0.785195   0.21480502]]
[[0.14491174 0.85508835]]
[[0.12174413 0.87825584]]
[[0.0012889 0.9987111]]
[[0.00124201 0.99875796]]
[[0.00175196 0.9982481 ]]
[[0.00366398 0.996336  ]]
[[0.00172821 0.9982718 ]]
[[0.01330553 0.98669446]]
[[0.00107434 0.9989256 ]]
[[0.08820377 0.9117962 ]]
[[0.00146082 0.9985392 ]]
[[0.00179833 0.9982016 ]]
[[0.00236019 0.9976398 ]]
[[0.08083125 0.9191688 ]]
[[0.00239978 0.99760026]]
[[0.00150382 0.9984962 ]]
[[0.00196975 0.99803025]]
[[0.00161531 0.9983847 ]]
[[0.00450154 0.99549854]]
[[0.00113967 0.99886036]]
[[

[[0.0075142  0.99248576]]
[[0.00108033 0.9989197 ]]
[[0.00099966 0.9990004 ]]
[[0.00109722 0.9989028 ]]
[[0.04859449 0.9514055 ]]
[[0.00184837 0.9981516 ]]
[[0.00382996 0.99617   ]]
[[0.00157108 0.9984289 ]]
[[0.02138964 0.9786104 ]]
[[0.00139881 0.99860126]]
[[0.00196369 0.9980363 ]]
[[0.0133385 0.9866615]]
[[9.5788605e-04 9.9904209e-01]]
[[0.00645302 0.993547  ]]
[[0.1638154 0.8361846]]
[[0.00450154 0.99549854]]
[[0.07785629 0.92214376]]
[[0.00341738 0.9965827 ]]
[[0.00152381 0.9984762 ]]
[[0.00573431 0.99426574]]
[[0.00184921 0.99815077]]
[[0.00200257 0.99799746]]
[[0.00252139 0.99747866]]
[[0.0060938 0.9939062]]
[[0.03802305 0.9619769 ]]
[[0.03496031 0.96503973]]
[[0.00154793 0.99845207]]
[[0.0053652  0.99463487]]
[[0.00250878 0.99749124]]
[[0.00307884 0.9969212 ]]
[[0.00370756 0.9962924 ]]
[[0.00222463 0.9977754 ]]
[[0.00124581 0.99875414]]
[[0.00350081 0.9964992 ]]
[[0.00752345 0.9924765 ]]
[[0.05958289 0.9404171 ]]
[[0.00170116 0.9982988 ]]
[[0.09126231 0.9087377 ]]
[[0.00235731

[[0.00905445 0.9909456 ]]
[[0.36931708 0.63068295]]
[[0.00584373 0.99415624]]
[[0.00209892 0.9979011 ]]
[[0.0045462  0.99545383]]
[[0.10325652 0.8967434 ]]
[[0.02370958 0.97629035]]
[[0.01087588 0.98912406]]
[[0.0095036 0.9904964]]
[[0.00164281 0.9983571 ]]
[[0.00503134 0.9949687 ]]
[[0.06528413 0.9347159 ]]
[[0.00110598 0.99889404]]
[[0.01793912 0.9820609 ]]
[[0.00123635 0.9987637 ]]
[[0.07713611 0.92286396]]
[[0.00264533 0.99735475]]
[[0.00297783 0.9970222 ]]
[[0.0098905  0.99010956]]
[[0.00270684 0.9972932 ]]
[[0.00351541 0.99648464]]
[[8.629029e-04 9.991371e-01]]
[[0.01028279 0.98971725]]
[[0.02293138 0.9770686 ]]
[[0.0049281 0.9950718]]
[[0.00243412 0.99756587]]
[[0.00424176 0.99575824]]
[[0.04331575 0.95668423]]
[[9.323881e-04 9.990676e-01]]
[[0.0020562 0.9979438]]
[[0.00197606 0.998024  ]]
[[0.00379944 0.9962005 ]]
[[0.00468564 0.99531436]]
[[0.00297719 0.9970228 ]]
[[0.02063784 0.97936213]]
[[0.00243568 0.9975643 ]]
[[0.00239864 0.99760133]]
[[0.00408506 0.9959149 ]]
[[0.018561

[[0.00274483 0.99725515]]
[[0.00200558 0.9979944 ]]
[[0.05971742 0.9402826 ]]
[[0.00155745 0.9984426 ]]
[[0.12117925 0.8788208 ]]
[[0.00108766 0.99891233]]
[[0.01608028 0.9839197 ]]
[[0.11096937 0.8890307 ]]
[[9.905230e-04 9.990095e-01]]
[[0.00197615 0.99802387]]
[[0.00450154 0.99549854]]
[[0.03125782 0.9687422 ]]
[[6.8120199e-04 9.9931884e-01]]
[[0.00117611 0.9988238 ]]
[[0.00133401 0.99866605]]
[[0.13399836 0.86600167]]
[[0.00138797 0.99861205]]
[[0.00489763 0.9951024 ]]
[[0.00580077 0.9941993 ]]
[[0.00372267 0.9962773 ]]
[[0.00603652 0.9939634 ]]
[[0.01165571 0.9883443 ]]
[[0.07138529 0.92861474]]
[[0.04339834 0.9566016 ]]
[[0.0142062  0.98579377]]
[[0.00639949 0.9936005 ]]
[[0.00111359 0.9988864 ]]
[[9.8580669e-04 9.9901414e-01]]
[[0.00245671 0.99754333]]
[[0.00129885 0.9987011 ]]
[[0.00244903 0.997551  ]]
[[0.0018098 0.9981902]]
[[0.00577651 0.99422354]]
[[0.00451776 0.9954822 ]]
[[0.00119968 0.9988003 ]]
[[0.22047392 0.77952605]]
[[0.00127228 0.99872774]]
[[0.00238139 0.9976186 ]

...

[[0.00450154 0.99549854]]
[[0.01113232 0.9888677 ]]
[[0.00415442 0.9958456 ]]
[[0.00855284 0.9914472 ]]
[[0.01273792 0.98726207]]
[[0.00273473 0.99726534]]
[[0.11387106 0.88612896]]
[[0.00339501 0.99660504]]
[[0.00110981 0.9988902 ]]
[[0.01580059 0.9841994 ]]
[[0.00250906 0.997491  ]]
[[0.025095 0.974905]]
[[9.534327e-04 9.990465e-01]]
[[0.04476926 0.9552307 ]]
[[0.00268692 0.9973131 ]]
[[0.00113754 0.9988625 ]]
[[0.00401626 0.9959837 ]]
[[0.00118766 0.9988123 ]]
[[0.00190303 0.998097  ]]
[[0.00388014 0.99611986]]
[[0.00528562 0.9947143 ]]
[[8.7690749e-04 9.9912316e-01]]
[[0.00116251 0.99883753]]
[[0.11956609 0.8804339 ]]
[[0.00669707 0.99330294]]
[[0.00372417 0.9962759 ]]
[[0.02159717 0.97840285]]
[[0.00237521 0.9976248 ]]
[[0.03267519 0.96732485]]
[[0.00274073 0.99725926]]
[[0.00364028 0.99635977]]
[[0.00150326 0.9984968 ]]
[[0.00106865 0.99893135]]
[[0.00100402 0.99899596]]
[[0.00811282 0.99188715]]
[[0.00178574 0.99821424]]
[[0.04339123 0.95660883]]
[[0.00225621 0.99774384]]
[[0.00

[[0.00617737 0.9938226 ]]
[[0.00101674 0.9989832 ]]
[[0.01801109 0.9819889 ]]
[[0.03370722 0.9662928 ]]
[[0.056191 0.943809]]
[[0.00666829 0.99333173]]
[[0.00239205 0.99760795]]
[[0.00341076 0.9965893 ]]
[[0.00176538 0.9982346 ]]
[[0.00189821 0.9981018 ]]
[[0.0067993 0.9932007]]
[[0.01025766 0.9897424 ]]
[[0.0051427 0.9948573]]
[[0.00399046 0.99600947]]
[[0.007663   0.99233705]]
[[0.06587974 0.93412024]]
[[0.0298386 0.9701614]]
[[0.00317593 0.996824  ]]
[[0.00204557 0.99795437]]
[[0.00107625 0.9989237 ]]
[[0.00349961 0.9965004 ]]
[[0.00573746 0.9942625 ]]
[[0.00691574 0.9930843 ]]
[[0.00154406 0.9984559 ]]
[[0.00321182 0.99678814]]
[[0.01470531 0.9852947 ]]
[[0.00105215 0.9989479 ]]
[[0.03012499 0.969875  ]]
[[0.00689989 0.99310005]]
[[0.00621016 0.99378985]]
[[0.00658295 0.993417  ]]
[[0.00133808 0.9986619 ]]
[[0.00335934 0.9966407 ]]
[[0.13126004 0.86873996]]
[[0.00221962 0.9977804 ]]
[[0.06301439 0.9369856 ]]
[[0.05122607 0.94877386]]
[[0.0080045 0.9919956]]
[[0.00253714 0.99746287]

[[0.00161133 0.99838865]]
[[0.0232838  0.97671616]]
[[0.02669491 0.9733051 ]]
[[0.00150896 0.99849105]]
[[0.00499335 0.9950067 ]]
[[0.01999979 0.98000026]]
[[0.00505179 0.99494815]]
[[0.00927943 0.9907205 ]]
[[0.00985036 0.9901496 ]]
[[0.02399887 0.97600114]]
[[0.00168156 0.99831843]]
[[0.00326015 0.9967398 ]]
[[0.0014768 0.9985232]]
[[0.01777595 0.98222405]]
[[0.04432773 0.95567226]]
[[0.02219811 0.97780186]]
[[0.0037407  0.99625933]]
[[0.12578763 0.8742124 ]]
[[0.09688276 0.90311724]]
[[0.00141478 0.9985852 ]]
[[0.00162735 0.99837273]]
[[0.00338804 0.9966119 ]]
[[0.07593739 0.9240626 ]]
[[0.00319039 0.99680954]]
[[0.00173311 0.9982668 ]]
[[9.9238614e-04 9.9900764e-01]]
[[0.00384314 0.9961569 ]]
[[0.00336312 0.99663687]]
[[0.03245607 0.96754396]]
[[0.00769773 0.9923023 ]]
[[0.00391388 0.9960861 ]]
[[0.00273333 0.99726665]]
[[0.0020441 0.9979559]]
[[8.1158202e-04 9.9918836e-01]]
[[0.00236902 0.99763095]]
[[0.08229542 0.9177046 ]]
[[0.00572755 0.9942725 ]]
[[0.004568 0.995432]]
[[0.0011

[[0.01345857 0.9865414 ]]
[[0.00775244 0.9922476 ]]
[[0.00734762 0.9926523 ]]
[[0.00109037 0.9989096 ]]
[[0.00646599 0.99353397]]
[[0.00698948 0.99301046]]
[[0.00324704 0.9967529 ]]
[[0.00161682 0.99838316]]
[[0.00195865 0.9980413 ]]
[[9.85980e-04 9.99014e-01]]
[[0.01089109 0.9891089 ]]
[[0.00168603 0.998314  ]]
[[0.00639218 0.9936079 ]]
[[0.00159092 0.9984091 ]]
[[0.00316288 0.99683714]]
[[0.00175755 0.9982425 ]]
[[8.4715727e-04 9.9915290e-01]]
[[0.00700301 0.992997  ]]
[[0.11806035 0.88193965]]
[[0.00390562 0.9960944 ]]
[[0.00308572 0.99691427]]
[[0.00567779 0.9943222 ]]
[[0.00435025 0.99564976]]
[[0.00347695 0.996523  ]]
[[0.0056001  0.99439985]]
[[0.01441955 0.9855804 ]]
[[0.00651417 0.99348587]]
[[0.01540442 0.9845956 ]]
[[0.01439611 0.9856039 ]]
[[0.00488064 0.99511933]]
[[0.00832621 0.99167377]]
[[0.0019159  0.99808407]]
[[0.00659239 0.9934076 ]]
[[0.08012526 0.9198747 ]]
[[0.0270731 0.9729269]]
[[0.00344904 0.9965509 ]]
[[0.00148883 0.99851114]]
[[0.00121365 0.9987864 ]]
[[0.05

[[0.00273246 0.9972676 ]]
[[0.00432617 0.99567384]]
[[0.00562093 0.9943791 ]]
[[0.00235358 0.9976464 ]]
[[0.00456483 0.9954352 ]]
[[0.3299202  0.67007977]]
[[0.0090998  0.99090016]]
[[0.00318407 0.996816  ]]
[[0.00725235 0.9927476 ]]
[[0.05977474 0.94022524]]
[[0.00130396 0.998696  ]]
[[0.00127021 0.99872977]]
[[0.05265972 0.94734025]]
[[0.14500932 0.8549907 ]]
[[0.00714896 0.992851  ]]
[[0.00662427 0.9933757 ]]
[[0.00384845 0.9961515 ]]
[[0.00294024 0.99705976]]
[[0.00239292 0.9976071 ]]
[[0.00241579 0.9975842 ]]
[[0.00168646 0.99831355]]
[[0.00314509 0.9968549 ]]
[[8.815167e-04 9.991185e-01]]
[[0.02755207 0.97244793]]
[[0.00627336 0.9937266 ]]
[[0.00112914 0.9988708 ]]
[[0.01119402 0.988806  ]]
[[0.00185568 0.99814427]]
[[0.00228891 0.99771106]]
[[0.00244472 0.9975553 ]]
[[0.00106378 0.99893624]]
[[0.03022811 0.96977186]]
[[0.00667467 0.99332535]]
[[0.00111398 0.99888605]]
[[0.06563982 0.93436015]]
[[0.00437537 0.9956246 ]]
[[0.03626234 0.96373767]]
[[0.00774886 0.9922511 ]]
[[8.5335

In [23]:
preds

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [37]:
with open('out/prediction_2.csv', 'w') as f:
    f.write('Id,Prediction'+'\n')
    for i,p in zip(test_ids, preds):
        if str(p) == "0":
            p = "-1"
        f.write(i+','+str(p)+'\n')

In [22]:
preds_ok = [-1 if x == 0 else None for x in preds]
print(keras.metrics.accuracy(test_y, preds_ok))

ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.